# Convolutional Neural Networks (CNNs) with TensorFlow

This notebook aims to provide a practical introduction to the fundamental concepts used in Convolutional Neural Networks (CNNs). The focus will be on the convolution operation, the application of filters to detect image features, and the function of pooling operations like max pooling. Finally, this notebook demonstrates how to implement a complete CNN using TensorFlow (TF).

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

print("TensorFlow version:", tf.__version__)

Load a sample image


In [ ]:
#image = tf.keras.utils.get_file("image.jpg",
#    "https://ensit-edst.com/uploads/sliders/155aebede8668727eaef40e677f8102c.jpg")

image = tf.keras.utils.get_file("image.jpg",
    "https://media.istockphoto.com/id/531314246/photo/adenocarcinoma.jpg?s=612x612&w=0&k=20&c=y0jJSS8HmIoT93b03FUwy_-Bj0Dr5NzhWyatDrhO2kE=")

img = tf.keras.utils.load_img(image, target_size=(408,612))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

plt.imshow(img_array[0])
plt.title("Original Image")
plt.axis("off")
plt.show()


### Convolution
Convolution applies a filter (kernel) across the image to detect features such as edges, textures, or patterns.
- **Kernel**: a small matrix (e.g., 3×3) that slides over the image.
- **Operation**: multiply element‑wise and sum to produce a new pixel value.


#### Edge Detection
This kernel highlights regions where pixel intensity changes sharply.

In [ ]:
# Define a simple edge detection kernel
edge_kernel = np.array([[-1, -1, -1],
                        [-1,  8, -1],
                        [-1, -1, -1]], dtype=np.float32)

edge_kernel = edge_kernel.reshape((3,3,1,1))

# Convert image to grayscale
gray = tf.image.rgb_to_grayscale(img_array)

# Apply convolution
conv = tf.nn.conv2d(gray, edge_kernel, strides=[1,1,1,1], padding="SAME")

# Visualize the result
plt.imshow(conv[0,:,:,0], cmap="gray")
plt.title("Edge Detection via Convolution")
plt.axis("off")
plt.show()


#### Sharpening
This kernel enhances fine details and makes edges more pronounced.

In [ ]:
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]], dtype=np.float32)

sharpen_kernel = sharpen_kernel.reshape((3,3,1,1))

# Convert image to grayscale for simplicity
gray = tf.image.rgb_to_grayscale(img_array)

# Apply convolution
conv = tf.nn.conv2d(gray, sharpen_kernel, strides=[1,1,1,1], padding="SAME")

# Visualize the result
plt.imshow(conv[0,:,:,0], cmap="gray")
plt.title("Sharpened Image via Convolution")
plt.axis("off")
plt.show()


#### Blurring
This kernel smooths the image, reducing noise and detail.

In [ ]:
blur_kernel = np.ones((5,5), dtype=np.float32) / 25.0

blur_kernel = blur_kernel.reshape((5,5,1,1))

# Convert image to grayscale for simplicity
gray = tf.image.rgb_to_grayscale(img_array)

# Apply convolution
conv = tf.nn.conv2d(gray, blur_kernel, strides=[1,1,1,1], padding="SAME")

# Visualize the result
plt.imshow(conv[0,:,:,0], cmap="gray")
plt.title("Blurred Image via Convolution")
plt.axis("off")
plt.show()


#### Embossing
This kernel creates a 3D relief effect, making the image look raised or carved.


In [ ]:
emboss_kernel = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [0,   1, 2]], dtype=np.float32)

emboss_kernel = emboss_kernel.reshape((3,3,1,1))

# Convert image to grayscale for simplicity
gray = tf.image.rgb_to_grayscale(img_array)

# Apply convolution
conv = tf.nn.conv2d(gray, emboss_kernel, strides=[1,1,1,1], padding="SAME")

# Visualize the result
plt.imshow(conv[0,:,:,0], cmap="gray")
plt.title("Embossed Image via Convolution")
plt.axis("off")
plt.show()


### Pooling
Pooling reduces the spatial dimensions of an image while preserving important features.
- **Max Pooling**: keeps the maximum value in each region.
- **Average Pooling**: computes the average value in each region.
Pooling helps reduce computation and control overfitting.


In [ ]:
# Max pooling
max_pool = tf.nn.max_pool2d(gray, ksize=3, strides=2, padding="SAME")

# Average pooling
avg_pool = tf.nn.avg_pool2d(gray, ksize=3, strides=2, padding="SAME")

# Visualize the result
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(gray[0,:,:,0], cmap="gray")
plt.title("Original Grayscale")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(max_pool[0,:,:,0], cmap="gray")
plt.title("Max Pooling")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(avg_pool[0,:,:,0], cmap="gray")
plt.title("Average Pooling")
plt.axis("off")

plt.show()


### Defining the CNN Architecture

A typical CNN consists of alternating Convolutional (Conv2D) and Pooling (MaxPooling2D) layers, followed by a Flatten layer and Dense output layers.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np

# Define the CNN model architecture
cnn_model = Sequential([
    # 1. Convolutional Layer: Learns 32 feature maps (filters)
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    
    # 2. Pooling Layer: Reduces feature map size by half (28x28 -> 14x14)
    MaxPooling2D((2, 2)),
    
    # 3. Another Conv Layer: Learns 64 features
    Conv2D(64, (3, 3), activation='relu'),
    
    # 4. Another Pooling Layer: Reduces feature map size (14x14 -> 7x7)
    MaxPooling2D((2, 2)),
    
    # 5. Flatten Layer: Flattens the 7x7x64 3D output into a 1D vector (3136 elements)
    Flatten(),
    
    # 6. Dense Hidden Layer: Standard fully connected layer
    Dense(128, activation='relu'),
    
    # 7. Output Layer: 10 units for 10 classes (Fashion MNIST)
    Dense(10) # No activation here, as SparseCategoricalCrossentropy handles logits
])

# Display the model summary
cnn_model.summary()